In [1]:
import tensorflow as tf
import numpy as np

/Users/sumitbhanwala/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
## as tensor flow supports broadcasting
## so we can by default subtract and the test data will get subtracted from 
## every training Data 
def processImage(testDatapt, trainingDatapts):
    return trainingDatapts - testDatapt;

In [3]:
## reading data online other alternative can be is to store
## data locally and than read using queues and multithreading
from tensorflow.examples.tutorials.mnist import input_data
## each image has only one label here that is what is
## represented by one_hot = True variable
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
## loading the data here for the processing and then for the validation
trainingData, traininglabel = mnist.train.next_batch(10000)
validationData, validationLabel= mnist.test.next_batch(200);
## as each image is of 784 dimensions
trainingDatapts =  tf.placeholder("float", [None , 784]);
## keeping the shape still as none as we might end up feeding more 
## than one vector
testDatapt =  tf.placeholder("float", [784]); 

## processCurrentImage will be the L1 distance of current
## image with all the training data
processCurrentImage = processImage(testDatapt, trainingDatapts);
## making the absolute of the difference between the image
L1Imageabs = tf.abs(processCurrentImage);
## calucating the L1 distance which is essentially
## equal to sum of all the columns of a row
L1Distance = tf.reduce_sum(L1Imageabs, axis  = 1);

## getting the index of the training example which gave the 
## minimum distance and we will classify our image as the same
## label
imageIndex = tf.argmin(L1Distance, 0); 

## keeping the L1 distance for now but it can be modified 
## further to improve the results
init = tf.global_variables_initializer()
    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
with tf.Session() as sess:
    sess.run(init)
    correct = 0;
    for i in range(len(validationData)):
        currentData = validationData[i];
        currentLabel = validationLabel[i];
        ## starting the feed dictionary for the newly entered field
        ## and than we will find its minimum distance with all the Point
        ## and give the label the point have which gives the minimum distance
        feedDict = {testDatapt: currentData,trainingDatapts: trainingData }
        distance = sess.run(processCurrentImage , feedDict);
        distanceAbsolute = sess.run(L1Imageabs, {processCurrentImage: distance});
        reducedDistance = sess.run(L1Distance, {L1Imageabs: distanceAbsolute});
        imageIndexfinal = sess.run(imageIndex, {L1Distance: reducedDistance});
        # Calculate accuracy
        if np.argmax(traininglabel[imageIndexfinal]) == np.argmax(validationLabel[i]):
            correct+=1;
        ## getting each row of the training matrix notation
    print("Accuracy is",correct/len(validationData) * 100);
    writer = tf.summary.FileWriter('./TensorGraph', sess.graph)
    writer.close()
    

Accuracy is 94.5
